# Mining Big Datasets
### Assignment 1
The goal of this assignment is to implement a simple workflow that will assess the similarity between supermarket customers and suggest for any input customer a list of his/her 10 most similar other customers. In order to calculate the similarity between customers you will first have to compute the dissimilarity for every given attribute as discussed in lecture “Measuring Data Similarity”. In order to fulfill this assignment, you will have to perform the following tasks:

## 1) Import and pre-process the dataset with customers
You will download the groceries.csv dataset from moodle. This dataset contains demographic characteristics of supermarket customers along with a list of groceries. In specific the dataset includes 10000 supermarket customer profiles with the following attributes:
* Customer ID: The unique id of the customer.
* Age: The age of the customer.
* Sex: Male-Female.
* Marital Status: Married, Single, Divorced.
* Education: Primary, Secondary, Tertiary.
* Annual Income: The annual customer income.
* Customer Rating: The rating of the supermarket from the customer (Poor, Fair, Good, Very Good, Excellent).
* Persons in Household: Number of persons in the household.
* Occupation: The occupation of each customer (retired, housemaid, unemployed, management, entrepreneur, blue-collar, self-employed, services, technician).
* Groceries: A list of the customer groceries.

For any numerical missing values, you should replace them with the average value of the attribute (keeping the integer part of the average).

### Solution

In [2]:
# Importing packages
import numpy as np
import pandas as pd

In the first place, we are going to read the groceries.csv file and  take a look at a sample of the data.

In [46]:
groceries= pd.read_csv("D:\\Users\\astar\\Desktop\\supermarket\\groceries.csv",sep=";")
groceries.sample(10) 

,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
2994,2995,49,male,married,primary,5000,very_good,2,services,oil
5734,5735,,male,married,primary,27000,good,3,unemployed,"meat,whole milk,dessert,soft cheese,frozen mea..."
7775,7776,29,female,married,secondary,38000,fair,3,management,"chicken,grapes,whole milk,yogurt,sliced cheese..."
4707,4708,40,male,single,primary,30000,poor,2,blue-collar,canned beer
7234,7235,61,male,married,secondary,30000,good,3,self-employed,"sausage,domestic eggs,rolls/buns,canned fish,c..."
263,264,21,male,married,tertiary,36000,poor,3,unemployed,fruit/vegetable juice
9864,9865,30,female,married,tertiary,29000,fair,3,housemaid,"citrus fruit,root vegetables,other vegetables,..."
5751,5752,79,male,married,primary,32000,good,3,retired,"sausage,tropical fruit,root vegetables,other v..."
3803,3804,64,male,single,primary,31000,very_good,1,self-employed,"cat food,canned beer"
8949,8950,34,male,married,primary,45000,good,3,self-employed,"sausage,tropical fruit,grapes,berries,other ve..."


At this point, it is important to see the types of our data columns. It seems that numeric variables like "Age" and "Income" are object type and must be converted into numeric type.

In [4]:
groceries.dtypes

Customer_ID              int64
Age                     object
Sex                     object
Marital_Status          object
Education               object
Income                  object
Customer_Rating         object
Persons_in_Household     int64
Occupation              object
Groceries               object
dtype: object

After converting "Age" and "Income" columns into numeric type, let's check if the convertion was made succesfully.

In [47]:
groceries["Age"] = pd.to_numeric(groceries.Age, errors='coerce')
groceries["Income"] = pd.to_numeric(groceries.Income, errors='coerce')
groceries.dtypes

Customer_ID               int64
Age                     float64
Sex                      object
Marital_Status           object
Education                object
Income                  float64
Customer_Rating          object
Persons_in_Household      int64
Occupation               object
Groceries                object
dtype: object

Now , we are going to find if there are any missing values in the columns of the dataset.

In [49]:
miss_values= groceries.isnull().sum()
print(miss_values)

Customer_ID               0
Age                     473
Sex                       0
Marital_Status            0
Education                 0
Income                  477
Customer_Rating           0
Persons_in_Household      0
Occupation                0
Groceries                 0
dtype: int64


There are 473 mising values in  the "Age" column and 477 missing values in the "Income" column. These values  will be replaced by the average value (keeping the integer part of the average)  of the corresponding column.

In [65]:
mean_age = int(groceries['Age'].mean())
groceries['Age']=groceries['Age'].fillna(mean_age)
mean_income = int(groceries['Income'].mean())
groceries['Income']=groceries['Income'].fillna(mean_income)

## 2) Compute data (dis-)similarity
In order to measure the similarity between the supermarket customers you could form the dissimilarity matrix for all given attributes. As described in lecture “Measuring Data Similarity”, for every given attribute you first distinguish its type (categorical, ordinal, numerical or set) and then compute the dissimilarity of its values accordingly. For set similarity use the Jaccard similarity between sets. Then, you can calculate the average of the computed dissimilarities in order to form the dissimilarity over all attributes. Depending of the machine used to implement this assignment you should decide whether is feasible to compute the dissimilarity matrices or have the computations performed on-the-fly for a pair of customers.

### Solution

Firstly, we are going to distinguish the type of each attribute (categorical, ordinal, numerical or set) and write down the types needed for the calculation of the dissimilarity:

| Numerical Attributes    | Categorical Attributes      | Ordinal Attributes      |    Set Attributes    |
| --------------------    | ----------------------      | ------------------      | -------------------- |
| Age                     | Sex                         | Education               |       Groceries      |
| Income                  | Marital_Status              | Customer_Rating         |                      |
| Persons_in_Household    | Occupation                  |                         |                      |
|$$d (a,b)= \frac{|a-b|}{maxvalue-minvalue} $$ | $$d(a,b) = 1$$ if a≠b,0 otherwise | $$d (a,b)= \frac{| rank(a)-rank(b)|}{maxrank-minrank} $$| $$JaccardDis(S1,S2) = 1 - \frac{|S1 \cap S2|}{|S1 \cup S2|}$$ |

where a,b are attributes and S1,S2 are sets of attributes.

### calcDiss Function
The "calcDiss" Function calculates the dissimilarity matrix between the customer given and all the other customers of the dataset for a specific attribute. It is appropriate for categorical, numerical and ordinal attributes.

In [70]:
def calcDiss(data,column, index ):
    matrix = list()
    row = data[column][index]
    max = 1
    min  = 0
    for r in data[column]:
        cur_row = row
        loop_row = r
        dis = 0
        # we initialize the dis, min, max as a categorical attribute
        if cur_row == loop_row:
                dis = 0
        else:
                dis = 1
        #if it is an ordinal attribute, change max, min , dis       
        if data[column].name == 'Education' or data[column].name == 'Customer_Rating':
         #if it is the ordinal attribute named "Education" , change max, min ,
         #dis in the suitable value based on the type mentioned  
         if data[column].name == 'Education':
                dict = {
                     'primary':1,
                     'secondary':2,
                     'tertiary':3
                }
                cur_row = dict[row]
                loop_row = dict[r] 
                max = 3
                min  = 1
                dis = abs(cur_row - loop_row) / (max - min)
                
         #if it is the ordinal attribute named "Customer_Rating", change max, min ,
         #dis in the suitable value based on the type mentioned  
         else:
            
                dict = {
                     'poor':1, 
                     'fair':2,
                     'good':3, 
                     'very_good':4, 
                     'excellent':5
                }
                cur_row = dict[row]
                loop_row = dict[r]
                max = 5
                min  = 1
                dis = abs(cur_row - loop_row) / (max - min)
                
        #if it is a numeric attribute ("Age","Income","Persons_in_Household")  
        #change max, min, dis in the suitable value based on the type mentioned 
        elif data[column].name == 'Age' or data[column].name == 'Income' or data[column].name == 'Persons_in_Household': 
            max = data[column].max()
            min  = data[column].min()
            dis = abs(cur_row - loop_row) / (max - min)
            
           
        matrix.append(dis)
                                      
    return np.asarray(matrix)

### calcDissGroc Function
The "calcDissGroc" Function calculates the dissimilarity matrix between the customer given and all the other customers of the dataset for a specific attribute. It is appropriate for set attributes.

In [72]:
def calcDissGroc(data,index):
    matrix = list()
    row = data["Groceries"][index]
    for r in data["Groceries"]:
        #it splits every grocery list seperated by ,
        s1 = set(row.split(","))
        s2 = set(r.split(","))
        #it calculates jaccard disimilarity based on the type mentioned above
        jaccard_disimilarity = 1- (len(s1.intersection(s2)) / len(s1.union(s2)))
        matrix.append(jaccard_disimilarity);
        
    return np.asarray(matrix)

$$d(a,b)= \frac{1}{9}*(d_{Age}(a,b) + d_{Sex}(a,b) + d_{MaritalStatus}(a,b) + d_{Education}(a,b) + d_{Income}(a,b) + d_{CustomerRating}(a,b) + d_{PersonsInHousehold}(a,b) + d_{Occupation}(a,b) + d_{Groceries}(a,b)) $$


where a,b are attributes (Age, Sex, Marital_Status, Education, Income, Customer_Rating, Persons_in_Household, Occupation) or set of attrbutes (Groceries) . 

### calcDissAvg Function
Given an input customer, the "calcDissAvg" Function calculates the average of the dissimilarity values of all the attributes of each customer.

In [75]:
#we call the calcDiss and callDissGroc for the calculaiton 
#of Dissimilarity matrixs and then calculate the average of the attributes
def calcDissAvg(data, cust):
    index = cust - 1
    Age = calcDiss(data, 'Age', index)
    Sex = calcDiss(data,'Sex', index)
    Marital_Status = calcDiss(data, 'Marital_Status', index)
    Education =calcDiss(data,'Education',index)
    Income = calcDiss(data, 'Income', index)
    Customer_Rating = calcDiss(data,'Customer_Rating',index)
    Persons_in_Household= calcDiss(data, 'Persons_in_Household', index)
    Occupation = calcDiss(data, 'Occupation', index)
    Groceries = calcDissGroc(data,index)
    avg = ((Age + Sex + Marital_Status + Education + Income + Customer_Rating + Persons_in_Household + Occupation + Groceries)/9 ) 
    return avg 
    

## 3) Nearest Neighbor (NN) search
Using the dissimilarities computed as discussed in the previous step, you will calculate the 10-NN (most similar) customers for the customers with ids listed below:
73, 563, 1603, 2200, 3703, 4263, 5300, 6129, 7800, 8555
For this task your script must take as input the customer-id and return the list of her 10 nearest neighbors (most similar), along with the corresponding similarity score.
An example of the script output for customer id =1 follows:

#### 10 NN for Customer 1

|Customer ID | Similarity Score |
|----------- | -----------------|
|7749 | |
|7931 | |
|9514 | |
|628  | |
|6918 | |
|4230 | |
|3148 | |
|4647 | |
|2105 | |
|8050 | |

### Solution

### similarity Function
The "similarity" Function calculates the similarity score between the customer given and all the other customers of the dataset.

$$ similarityScore = 1- dissimilarityScore $$

In [74]:
def similarity(data, nearest, cust):
    index = cust - 1
    nearest_list = nearest.tolist()
       
    print("10 NN FOR CUSTOMER "+ str(cust))
    groc = []
    for i in nearest_list:
        if i == index:
         continue;
        calc_sim = 1- calcDissAvg(data, cust)[i]
        Customer_ID = i + 1
        Similarity_Score = round(calc_sim,2)
        groc.append((Customer_ID, Similarity_Score))
        cols=['Customer_ID','Similarity_Score']

        
    result = pd.DataFrame(groc, columns=cols)
    
    return result

We wiil calculate the 10 Nearest neighbours to customer 73 and the similarity score for each one.

In [76]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,73))[0:11],73)
r

10 NN FOR CUSTOMER 73


,Customer_ID,Similarity_Score
0,1846,0.88
1,1291,0.87
2,1203,0.86
3,5881,0.85
4,1627,0.85
5,3953,0.85
6,6904,0.84
7,5922,0.84
8,8881,0.84
9,3623,0.84


We wiil calculate the 10 Nearest neighbours to customer 563 and the similarity score for each one.

In [77]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,563))[0:11],563)
r

10 NN FOR CUSTOMER 563


,Customer_ID,Similarity_Score
0,3634,0.93
1,6168,0.91
2,2839,0.88
3,6196,0.88
4,2766,0.87
5,8108,0.87
6,559,0.87
7,6929,0.87
8,9578,0.87
9,7202,0.87


We wiil calculate the 10 Nearest neighbours to customer 1603 and the similarity score for each one.

In [78]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,1603))[0:11],1603)
r

10 NN FOR CUSTOMER 1603


,Customer_ID,Similarity_Score
0,7345,0.87
1,7335,0.86
2,568,0.85
3,109,0.85
4,4814,0.85
5,6751,0.84
6,4628,0.84
7,168,0.84
8,8591,0.83
9,6841,0.83


We wiil calculate the 10 Nearest neighbours to customer 2200 and the similarity score for each one.

In [79]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,2200))[0:11],2200)
r

10 NN FOR CUSTOMER 2200


,Customer_ID,Similarity_Score
0,403,0.86
1,7497,0.84
2,8884,0.82
3,6722,0.81
4,5160,0.81
5,3551,0.81
6,5330,0.80
7,4928,0.79
8,6942,0.79
9,2667,0.78


We wiil calculate the 10 Nearest neighbours to customer 3703 and the similarity score for each one.

In [81]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,3703))[0:11],3703)
r

10 NN FOR CUSTOMER 3703


,Customer_ID,Similarity_Score
0,9942,0.88
1,1604,0.87
2,4838,0.86
3,3352,0.86
4,1837,0.86
5,3990,0.86
6,7194,0.85
7,7784,0.85
8,374,0.85
9,6793,0.85


We wiil calculate the 10 Nearest neighbours to customer 4263 and the similarity score for each one.

In [82]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,4263))[0:11],4263)
r

10 NN FOR CUSTOMER 4263


,Customer_ID,Similarity_Score
0,9536,0.88
1,4990,0.86
2,9051,0.86
3,2195,0.86
4,5829,0.84
5,3822,0.84
6,6183,0.84
7,5427,0.84
8,1896,0.83
9,5755,0.83


We wiil calculate the 10 Nearest neighbours to customer 5300 and the similarity score for each one.

In [83]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,5300))[0:11],5300)
r

10 NN FOR CUSTOMER 5300


,Customer_ID,Similarity_Score
0,8497,0.87
1,8982,0.87
2,8711,0.87
3,2110,0.87
4,7457,0.87
5,3533,0.86
6,3470,0.86
7,8068,0.86
8,1999,0.86
9,8905,0.86


We wiil calculate the 10 Nearest neighbours to customer 6129 and the similarity score for each one.

In [84]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,6129))[0:11],6129)
r

10 NN FOR CUSTOMER 6129


,Customer_ID,Similarity_Score
0,1082,0.89
1,6303,0.88
2,2029,0.87
3,7563,0.87
4,4933,0.87
5,6387,0.87
6,7870,0.87
7,7557,0.87
8,5680,0.86
9,5301,0.86


We wiil calculate the 10 Nearest neighbours to customer 7800 and the similarity score for each one.

In [85]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,7800))[0:11],7800)
r

10 NN FOR CUSTOMER 7800


,Customer_ID,Similarity_Score
0,2126,0.88
1,186,0.88
2,7470,0.86
3,2342,0.84
4,9116,0.83
5,673,0.83
6,8293,0.82
7,8212,0.82
8,1251,0.82
9,1847,0.81


We wiil calculate the 10 Nearest neighbours to customer 8555 and the similarity score for each one.

In [86]:
r = similarity(groceries, np.argsort(calcDissAvg(groceries,8555))[0:11],8555)
r

10 NN FOR CUSTOMER 8555


,Customer_ID,Similarity_Score
0,1486,0.88
1,6092,0.88
2,8732,0.87
3,3012,0.87
4,6823,0.87
5,3320,0.86
6,2691,0.86
7,4406,0.86
8,9336,0.86
9,3894,0.86
